<a href="https://colab.research.google.com/github/samitha278/multilayer-perceptron-char-lm/blob/main/mlp_char_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [76]:
from google.colab import files

uploaded = files.upload()


Saving names.txt to names (2).txt


In [77]:
words = open("names.txt","r").read().splitlines()

In [78]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

### Building dataset

In [79]:
block_size = 3


xs = []
ys = []


context = [0]*block_size



for w in words:

  for ch in w + '.':

    xs.append(context)
    next = stoi[ch]
    ys.append(next)


    context = context[1:] + [next]


x = torch.tensor(xs)
y = torch.tensor(ys)

In [80]:
x.shape,y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [81]:
x[:5]

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])

### Embedding

In [82]:
g = torch.Generator().manual_seed(2147483647)

In [83]:
C = torch.randn((27,2),generator=g,requires_grad=True)
C

tensor([[ 1.5674, -0.2373],
        [-0.0274, -1.1008],
        [ 0.2859, -0.0296],
        [-1.5471,  0.6049],
        [ 0.0791,  0.9046],
        [-0.4713,  0.7868],
        [-0.3284, -0.4330],
        [ 1.3729,  2.9334],
        [ 1.5618, -1.6261],
        [ 0.6772, -0.8404],
        [ 0.9849, -0.1484],
        [-1.4795,  0.4483],
        [-0.0707,  2.4968],
        [ 2.4448, -0.6701],
        [-1.2199,  0.3031],
        [-1.0725,  0.7276],
        [ 0.0511,  1.3095],
        [-0.8022, -0.8504],
        [-1.8068,  1.2523],
        [ 0.1476, -1.0006],
        [-0.5030, -1.0660],
        [ 0.8480,  2.0275],
        [-0.1158, -1.2078],
        [-1.0406, -1.5367],
        [-0.5132,  0.2961],
        [-1.4904, -0.2838],
        [ 0.2569,  0.2130]], requires_grad=True)

In [84]:
C[5]

tensor([-0.4713,  0.7868], grad_fn=<SelectBackward0>)

In [85]:
C[torch.tensor([2,3,5,5,5])]

tensor([[ 0.2859, -0.0296],
        [-1.5471,  0.6049],
        [-0.4713,  0.7868],
        [-0.4713,  0.7868],
        [-0.4713,  0.7868]], grad_fn=<IndexBackward0>)

In [86]:
C[torch.tensor([[2,3],[3,2],[5,5],[2,5]])]

tensor([[[ 0.2859, -0.0296],
         [-1.5471,  0.6049]],

        [[-1.5471,  0.6049],
         [ 0.2859, -0.0296]],

        [[-0.4713,  0.7868],
         [-0.4713,  0.7868]],

        [[ 0.2859, -0.0296],
         [-0.4713,  0.7868]]], grad_fn=<IndexBackward0>)

In [87]:
emb = C[x]
emb.shape

torch.Size([228146, 3, 2])

In [88]:
x[1],C[[0,5]],emb[1]

(tensor([0, 0, 5]),
 tensor([[ 1.5674, -0.2373],
         [-0.4713,  0.7868]], grad_fn=<IndexBackward0>),
 tensor([[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [-0.4713,  0.7868]], grad_fn=<SelectBackward0>))

### Preparing input neurons to feed hidden layer

In [89]:
emb[:5]

tensor([[[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [ 1.5674, -0.2373]],

        [[ 1.5674, -0.2373],
         [ 1.5674, -0.2373],
         [-0.4713,  0.7868]],

        [[ 1.5674, -0.2373],
         [-0.4713,  0.7868],
         [ 2.4448, -0.6701]],

        [[-0.4713,  0.7868],
         [ 2.4448, -0.6701],
         [ 2.4448, -0.6701]],

        [[ 2.4448, -0.6701],
         [ 2.4448, -0.6701],
         [-0.0274, -1.1008]]], grad_fn=<SliceBackward0>)

In [90]:
emb[:5,0]

tensor([[ 1.5674, -0.2373],
        [ 1.5674, -0.2373],
        [ 1.5674, -0.2373],
        [-0.4713,  0.7868],
        [ 2.4448, -0.6701]], grad_fn=<SelectBackward0>)

In [91]:
torch.cat([emb[:5,0],emb[:5,1],emb[:5,2]],dim=1)

tensor([[ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.4713,  0.7868],
        [ 1.5674, -0.2373, -0.4713,  0.7868,  2.4448, -0.6701],
        [-0.4713,  0.7868,  2.4448, -0.6701,  2.4448, -0.6701],
        [ 2.4448, -0.6701,  2.4448, -0.6701, -0.0274, -1.1008]],
       grad_fn=<CatBackward0>)

In [92]:
torch.cat([emb[:,0],emb[:,1],emb[:,2]],dim=1).shape

torch.Size([228146, 6])

In [93]:
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([228146, 6])

### torch view and storage

In [94]:
a = torch.arange(12)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [95]:
a.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

In [96]:
a.view(3,4)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [97]:
#only view has been changed memory allocation of a is same as before

# https://blog.ezyang.com/2019/05/pytorch-internals/

a.view(3,4).storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

### Implementing hidden layer

In [98]:
W1 = torch.randn((6,100),generator=g,requires_grad=True)
b1 = torch.randn(100,generator=g,requires_grad=True)

In [99]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [100]:
h.shape , h

(torch.Size([228146, 100]),
 tensor([[-0.9348,  1.0000,  0.9258,  ...,  0.9786, -0.1926,  0.9515],
         [ 0.2797,  0.9997,  0.7675,  ...,  0.9929,  0.9992,  0.9981],
         [-0.9960,  1.0000, -0.8694,  ..., -0.5159, -1.0000, -0.0069],
         ...,
         [-0.4849,  0.9972, -0.6418,  ..., -0.9641,  0.9996,  0.9873],
         [-0.9318,  0.9926, -0.9841,  ..., -0.8989, -0.9938,  0.5930],
         [-0.9736,  0.3844, -0.8744,  ..., -0.5093,  0.9998, -0.9975]],
        grad_fn=<TanhBackward0>))

### Output layer

In [101]:
W2 = torch.randn((100,27),generator=g,requires_grad=True)
b2 = torch.randn(27,generator=g,requires_grad=True)

In [102]:
logits = h @ W2 + b2
logits.shape

torch.Size([228146, 27])

### Softmax

In [103]:
p = logits.exp()
p /= p.sum(1,keepdim=True)

p.shape

torch.Size([228146, 27])

### Loss

In [104]:
loss = -p[torch.arange(p.shape[0]),y].log().mean()
loss

tensor(19.2358, grad_fn=<NegBackward0>)

### Efficient way to calculate Loss

In [105]:
loss = F.cross_entropy(logits,y)
loss

tensor(19.2358, grad_fn=<NllLossBackward0>)

### Backpropagation

In [106]:
parameters = [C,W1,b1,W2,b2]

In [107]:
for p in parameters:

  #p.requires_grad = True
  #don't set requires_grad here → must be set at init time,
  #otherwise loss won't track this param in the graph

  #fixed it , remember it

  p.grad = None

In [108]:
loss.backward()

## What’s happening under the hood?

* When you create a parameter with requires_grad=True, PyTorch starts tracking
every operation involving it and builds a computation graph.

* During loss.backward(), PyTorch walks backward through this graph and fills in p.grad for each parameter.

* Gradients accumulate in .grad (they don’t overwrite). That’s why we set p.grad = None before each backward pass.

* If you accidentally set requires_grad=True after the forward pass, the computation graph was already built without tracking, so no grads flow → loss.backward() fails.

In [115]:
for p in parameters:
  p.data -= 0.01 * p.grad

### Forward pass

In [116]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1 )
logits = h @ W2 +b2
loss = F.cross_entropy(logits,y)
loss                  #decreased liltle bit

tensor(19.1409, grad_fn=<NllLossBackward0>)